* Selector => Tag, ID  <br> (#아이디), CLASS(.클래스명)
* [name = 'asdf'] <br>
* [name ^= 'asdfasd'] <br>
* [name $= 'asdfasd'] <br>
* div p => find_all(div 찾고, 자손중에 p) <br>
* div > p => find_all(recursive = False) <br>
* div 찾고, 자식 p <br>
* div + p => find(div).find_next_sibling(P) <br>

In [1]:
import requests
from bs4 import BeautifulSoup
header = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36'}

In [2]:
def getDownload(url, param = None, retries = 3):
    resp = None
    try:
        resp = requests.get(url, params = param, headers = header)
        resp.raise_for_status()
    except requests.exceptions.HTTPError as e:
        if 500 <= resp.status_code < 600 and retries > 0:
            print('Retries : {0}'.format(retries))
            return getDownload(url, param, retries -1)
        else:
            print(resp.status_code)
            print(resp.reason)
            print(resp.request.headers)
            
    return resp

In [3]:
url = 'https://www.naver.com/'
html = getDownload(url)
dom = BeautifulSoup(html.text, 'lxml')

for tag in dom.select('.ah_list.PM_CL_realtimeKeyword_list_base .ah_k'):
    print(tag.text)

명륜진사갈비
카더가든
플로리다 프로젝트
노비스 비타주서
otp 뜻
기안84
프로듀스 x101 순위
런닝맨 머니게임
msi
보이스 시즌3
조수현
스페인하숙 재방송
나혼자산다 재방송
이마트 오픈시간
김성수
배우조수현
구해줘2
호랑이형님
박선호
김요한


## http://example.webscraping.com/places/default/index (Test Site)

In [ ]:
seed = 'http://example.webscraping.com/places/default/index'
html = getDownload(seed)
dom = BeautifulSoup(html.content, 'lxml')

In [ ]:
def urlExtractor(seed):
    html = getDownload(seed)
    dom = BeautifulSoup(html.text, 'lxml')
    
    links = [_['href'] for _ in dom.select('a')]
    
    unseen = list()
    
    for link in links:
        if link[0] == '/':
            newLink = requests.compat.urljoin(seed, link)
            if newLink not in unseen:
                unseen.append(newLink)
            else:
                print('Skipped2', link)
        else:
            print('Skipped1', link)
            
    print('{0}[{1}]'.format(seed, len(unseen)))
            
    return unseen

In [ ]:
import time
import random

queue = urlExtractor(seed)
seen = list()

while queue:
    seed = queue.pop(0)
    seen.append(seed)
    time.sleep(random.randint(1,3))
    links = urlExtractor(seed)
    # links => unseen 겹치면 안되고, seen에 있으면 안됨
    queue.extend([_ for _ in links if _ not in queue and _ not in seen])

In [4]:
html = getDownload('https://www.google.com/search', {'q':'박보영'})
dom = BeautifulSoup(html.content, 'lxml')

In [5]:
queue = [_['href'] for _ in dom.select('.r > a')]


In [6]:
for link in queue:
    html = getDownload(link)
    dom = BeautifulSoup(html.content, 'lxml')
    print(link, len([_['href'] for _ in dom.select('a') if _.has_attr('href')]))

https://ko.wikipedia.org/wiki/%EB%B0%95%EB%B3%B4%EC%98%81 1073
https://namu.wiki/w/%EB%B0%95%EB%B3%B4%EC%98%81 1285
https://www.msn.com/ko-kr/entertainment/news/%EB%B0%95%EB%B3%B4%EC%98%81-tvn-%EB%93%9C%EB%9D%BC%EB%A7%88-%EC%96%B4%EB%B9%84%EC%8A%A4-%EC%97%AC%EC%A3%BC%EC%9D%B8%EA%B3%B5/ar-BBLZzVh 59
https://twitter.com/hashtag/%EB%B0%95%EB%B3%B4%EC%98%81 227
https://news.joins.com/article/22895953 52
https://www.mk.co.kr/star/hot-issues/view/2019/05/293931/ 336
https://www.mk.co.kr/news/business/view/2019/03/181764/ 297
https://www.hankyung.com/entertainment/article/201905077647H 190


In [7]:
threshold = 3

def urlExtractor(seed, limit):
    if limit > threshold:
        return []
    
    html = getDownload(seed)
    dom = BeautifulSoup(html.text, 'lxml')
    
    links = [_['href'] for _ in dom.select('a') if _.has_attr('href')]
    
    unseen = list()
    
    for link in links:
        if len(link) > 1 and link[0] == '/':
            newLink = requests.compat.urljoin(seed, link)
            if newLink not in unseen:
                unseen.append((newLink, limit+1))
#             else:
#                 print('Skipped2', link)
                
        elif link.startswith('http'):
            unseen.append((link, limit+1))
#         else:
#             print('Skipped1', link)
            
#     print('{0}[{1}]'.format(seed, len(unseen)))
            
    return unseen

In [8]:
html = getDownload('https://www.google.com/search', {'q':'박보영'})
dom = BeautifulSoup(html.content, 'lxml')

In [9]:
queue = [(_['href'], 0) for _ in dom.select('.r > a')]


In [10]:
seen = list()

while queue:
    link = queue.pop(0)
    seen.append(link[0])
    links = urlExtractor(link[0], link[1])

    queueURL = [_[0] for _ in queue]
    queue.extend([_ for _ in links if _[0] not in queueURL and _ not in seen])
    
    print('{0} {1} [{2}]'.format('>' * link[1], link[0], len(links)))

 https://ko.wikipedia.org/wiki/%EB%B0%95%EB%B3%B4%EC%98%81 [820]
 https://namu.wiki/w/%EB%B0%95%EB%B3%B4%EC%98%81 [1139]
 https://www.msn.com/ko-kr/entertainment/news/%EB%B0%95%EB%B3%B4%EC%98%81-tvn-%EB%93%9C%EB%9D%BC%EB%A7%88-%EC%96%B4%EB%B9%84%EC%8A%A4-%EC%97%AC%EC%A3%BC%EC%9D%B8%EA%B3%B5/ar-BBLZzVh [47]
 https://twitter.com/hashtag/%EB%B0%95%EB%B3%B4%EC%98%81 [172]
 https://news.joins.com/article/22895953 [34]
 https://www.mk.co.kr/star/hot-issues/view/2019/05/293931/ [72]
 https://www.mk.co.kr/news/business/view/2019/03/181764/ [110]
 https://www.hankyung.com/entertainment/article/201905077647H [185]


KeyboardInterrupt: 